## Install Julia
Run cell then reload

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

### Using GPU

In [ ]:
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

# Assignment 1 Question 2
` l’aide de la librairie StochasticPrograms.jl, r ́esolvez num ́eriquement le probl`eme comme un programme stochastique `a deux  ́etapes, en faisant varier le nombre de sc ́enarios, tir ́es al ́eatoirement, de 5 `a 10000. Plus pr ́ecis ́ement, on demande de consid ́erer 5, 10, 100, 1000 et 10000 sc ́enarios.

Solving the following stochastic program:

\begin{align*}
    \min_{x} \quad & 2 x + E_ξ[Q(x,ξ)] \\
    \mbox{s.t. } & x \geq 0 \\
\end{align*}
where
\begin{align*}
Q(x,ξ) = \min_y\ & \xi y \\
\mbox{s.t. } & y \geq 1-x \\
             &y \geq 0 \\
\end{align*}
with
\begin{align*}
p_{\xi}(1) = 3/4 \qquad p_{\xi}(3) = 3/4
\end{align*}

In [1]:
import Pkg
Pkg.add("StochasticPrograms")
Pkg.add("HiGHS")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using StochasticPrograms
using HiGHS

In [3]:
using BenchmarkTools

## Setup


### Defining the stochastic program

Generating scenarios __CHECK RAND GENERATOR__

In [4]:
function generate_scenarios(num_scenarios, min_scenario=5, max_scenario=10000)
    prob = 1/num_scenarios
    scenarios = [@scenario ξ = rand(min_scenario:max_scenario) probability = prob for _ in 1:num_scenarios]
    return scenarios
end

generate_scenarios (generic function with 3 methods)

## Question 1 - Model

In [21]:

@stochastic_model q1_model begin

    @stage 1 begin
        @parameters begin
            cost = 2.0
            up_bound = 10000.0
        end
        @decision(q1_model, x >= 0)   # equivalent to @variable
        @constraint(q1_model, x <= up_bound)
        @objective(q1_model, Min, cost*x)
    end
    @stage 2 begin
        @uncertain ξ
        @recourse(q1_model,  y >= 0)   # equivalent to @decision
        @objective(q1_model, Min, ξ*y)
        @constraint(q1_model,  y >= 1-x)
    end
end

Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


## Newsvendor Model - Birge et Louveaux

In [ ]:
@stochastic_model simple_model begin
    @stage 1 begin
        @decision(simple_model, x >= 0)
        @objective(simple_model, Min, 0*x)
    end
    @stage 2 begin
        @known(simple_model, x)
        @uncertain ξ
        @recourse(simple_model, 0 <= y[i in 1:2])
        @objective(simple_model, Min, y[1] + y[2])
        @constraint(simple_model, y[1] - y[2] == ξ - x)
    end
end

In [36]:
function avg_y(model, nb_scenarios)
    optimal_y = model[2,:y]
    y_list = [value(optimal_y, i) for i in 1:nb_scenarios]
    y = sum(y_list)
    return y/nb_scenarios
end

avg_y (generic function with 1 method)

## Optimize Model for 5 Scenarios

In [40]:
res = Vector{Vector{Float64}}()
nb_scenarios = [5, 10, 100, 1000, 10000]
model  =
for n in nb_scenarios
    memory_used = @allocated begin

        ξ = generate_scenarios(n)

        ## LShape
        q2_highs = instantiate(q1_model, ξ, optimizer = LShaped.Optimizer)
        set_optimizer_attribute(q2_highs, MasterOptimizer(), HiGHS.Optimizer)
        set_optimizer_attribute(q2_highs, SubProblemOptimizer(), HiGHS.Optimizer)
        optimize!(q2_highs)
    end

    # Check the status of the optimization
    status = termination_status(q2_highs)

    if status == MOI.OPTIMAL
        # Retrieve the optimal solution
        optimal_objective = objective_value(q2_highs)
        optimal_x = optimal_decision(q2_highs)[1]
        avg_optimal_y =  avg_y(q2_highs, n)

        # Retrieve the runtime
        runtime = q2_highs.:optimizer.:optimizer.:solve_time

        # Store the results
        curr_res = [n, optimal_x, optimal_objective, avg_optimal_y, runtime, memory_used]
        push!(res, curr_res)

    else
        println("Optimization did not converge to an optimal solution.")
    end

end

L-Shaped Gap  (thresh = 1e-06, value = 0.9999)
  Objective:       20000.0
  Gap:             0.9998999999999951
  Number of cuts:  10






L-Shaped Gap  Time: 0:00:00 (3 iterations)
  Objective:       2.0
  Gap:             0.0
  Number of cuts:  10
  Iterations:      3


Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-2); columns 0(-2); elements 0(-3) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  8.1817390426e+02
HiGHS run time      :          0.00
Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-2); columns 0(-2); elements 0(-3) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  1.7269599022e+03
HiGHS run time      :          0.00
Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 

L-Shaped Gap  (thresh = 1e-06, value = 0.9999)
  Objective:       20000.0
  Gap:             0.9998999999999951
  Number of cuts:  20






L-Shaped Gap  Time: 0:00:00 (3 iterations)
  Objective:       2.0
  Gap:             0.0
  Number of cuts:  20
  Iterations:      3


...

Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  0.0000000000e+00






L-Shaped Gap  Time: 0:00:01 (3 iterations)
  Objective:       2.0
  Gap:             0.0
  Number of cuts:  2000
  Iterations:      3


Optimal Solution: 1000
x* = 1.0
Objective Value = 2.0
avg_optimal_y0.0
runtime1.7047138214111328
165516536
Int64Float64Float64
y Float64
Float64
Int64
Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-2); columns 0(-2); elements 0(-3) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  9.0157412223e+01
HiGHS run time      :          0.00
Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-2); columns 0(-2); elements 0(-3) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  4.7079813352e+02
HiGHS run time      :        

L-Shaped Gap  (thresh = 1e-06, value = 0.9999)
  Objective:       20000.0
  Gap:             0.9998999999999951
  Number of cuts:  20000


Excessive output truncated after 535981 bytes.

In [48]:
res

5-element Vector{Vector{Float64}}:
 [5.0, 1.0, 2.0, 0.0, 0.02350020408630371, 917032.0]
 [10.0, 1.0, 2.0, 0.0, 0.027155160903930664, 1.675072e6]
 [100.0, 1.0, 2.0, 0.0, 0.18802118301391602, 1.4733096e7]
 [1000.0, 1.0, 2.0, 0.0, 1.7047138214111328, 1.65516536e8]
 [10000.0, 1.0, 2.0, 0.0, 28.17290496826172, 3.459218288e9]